In [185]:
%pylab inline
%matplotlib inline

import pandas as pd
import numpy as np
import mmap
import time
import re

import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

import cufflinks as cf
import plotly.plotly as py
py.sign_in('albertoueda', '6yu6572hk6')
cf.set_config_file(world_readable=True,offline=False)

from unidecode import unidecode
UTF8 = 'utf-8'        

class Dict(dict):
    def __missing__(self, key):
            return False

#reload(sys)  # Solve the acentos problem, but create the terminal output problem
#sys.setdefaultencoding(UTF8)
# mypath = '/mnt/hd0/alberto/Dropbox/ufmg-not-code/datasets/dblp/cn-infocom-clusters/' # UFMG
# mypath = '/home/alberto/Dropbox/ufmg-not-code/datasets/dblp/cn-infocom-clusters/' # Home

Populating the interactive namespace from numpy and matplotlib


# Qualis

In [258]:
qualis = pd.read_csv('data/qualis-2013-cs-all-venues.csv')
qualis.Sigla = qualis.Sigla.str.upper()
qualis.head(5)

,Sigla,Índice-­‐H,Estrato
0,3DIM,29,B1
1,3DUI,18,B2
2,3PGCIC,4,B4
3,AAAI,93,A1
4,AAECC,17,B2


# Information Retrieval

In [168]:
venues = pd.read_csv('data/cs-ir-conf-all-years.tsv', sep='\t')
venues[['Shortname', 'Name']] = venues.Conferences.str.split(' - ', n=1, expand=True)
venues.Shortname = venues.Shortname.str.upper()
venues.head()

,Conferences,Publications,Citation,Shortname,Name
0,SIGIR - Research and Development in Informatio...,2717,72116,SIGIR,Research and Development in Information Retrieval
1,CIKM - International Conference on Information...,2636,28621,CIKM,International Conference on Information and Kn...
2,TREC - Text REtrieval Conference,1433,23782,TREC,Text REtrieval Conference
3,JCDL - ACM/IEEE Joint Conference on Digital Li...,1269,7203,JCDL,ACM/IEEE Joint Conference on Digital Libraries
4,DL - Digital Libraries,351,6546,DL,Digital Libraries


In [176]:
# Specific Assessments
venues.loc[venues.Shortname == 'ECDL', ['Shortname', 'Name']] = ['TPDL', 'Theory and Practice of Digital Libraries (aka ECDL)']

,Conferences,Publications,Citation,Shortname,Name
5,ECDL - European Conference on Digital Libraries,841,4765,TPDL,Theory and Practice of Digital Libraries (aka ...


In [264]:
v = venues.merge(qualis, left_on='Shortname', right_on='Sigla', how='left')
v = v.sort_values(by='Citation', ascending=False)
v.head(5)

,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato
0,SIGIR - Research and Development in Informatio...,2717,72116,SIGIR,Research and Development in Information Retrieval,SIGIR,122,A1
1,CIKM - International Conference on Information...,2636,28621,CIKM,International Conference on Information and Kn...,CIKM,85,A1
2,TREC - Text REtrieval Conference,1433,23782,TREC,Text REtrieval Conference,TREC,51,A2
3,JCDL - ACM/IEEE Joint Conference on Digital Li...,1269,7203,JCDL,ACM/IEEE Joint Conference on Digital Libraries,JCDL,53,A2
4,DL - Digital Libraries,351,6546,DL,Digital Libraries,DL,36,B1


In [295]:
v['ShortnameQualis'] = v.Shortname + ' (' + v.Estrato + ')'
v.loc[v['ShortnameQualis'].isnull(), 'ShortnameQualis'] = v['Shortname']

,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,SIGIR - Research and Development in Informatio...,2717,72116,SIGIR,Research and Development in Information Retrieval,SIGIR,122,A1,SIGIR (A1)
1,CIKM - International Conference on Information...,2636,28621,CIKM,International Conference on Information and Kn...,CIKM,85,A1,CIKM (A1)
2,TREC - Text REtrieval Conference,1433,23782,TREC,Text REtrieval Conference,TREC,51,A2,TREC (A2)
3,JCDL - ACM/IEEE Joint Conference on Digital Li...,1269,7203,JCDL,ACM/IEEE Joint Conference on Digital Libraries,JCDL,53,A2,JCDL (A2)
4,DL - Digital Libraries,351,6546,DL,Digital Libraries,DL,36,B1,DL (B1)
5,ECDL - European Conference on Digital Libraries,841,4765,TPDL,Theory and Practice of Digital Libraries (aka ...,TPDL,34,B1,TPDL (B1)
6,CLEF - Cross-Language Evaluation Forum,1059,4087,CLEF,Cross-Language Evaluation Forum,CLEF,16,B2,CLEF (B2)
7,RIAO - Recherche d'Information Assistee par Or...,574,3551,RIAO,Recherche d'Information Assistee par Ordinateur,NaN,NaN,NaN,RIAO
8,ECIR - European Colloquium on IR Research,686,3548,ECIR,European Colloquium on IR Research,ECIR,39,A2,ECIR (A2)
9,Multimedia Information Retrieval,248,2847,MULTIMEDIA INFORMATION RETRIEVAL,None,NaN,NaN,NaN,MULTIMEDIA INFORMATION RETRIEVAL


In [296]:
print('Found', len(v[v.Estrato.notnull()]), 'of', len(venues), 'venues')

Found 11 of 41 venues


In [297]:
v[v.Estrato.isnull()].sort_values(by='Citation', ascending=False).head()

,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
7,RIAO - Recherche d'Information Assistee par Or...,574,3551,RIAO,Recherche d'Information Assistee par Ordinateur,NaN,NaN,NaN,RIAO
9,Multimedia Information Retrieval,248,2847,MULTIMEDIA INFORMATION RETRIEVAL,None,NaN,NaN,NaN,MULTIMEDIA INFORMATION RETRIEVAL
11,Dublin Core Conference,65,1928,DUBLIN CORE CONFERENCE,None,NaN,NaN,NaN,DUBLIN CORE CONFERENCE
13,TRECVID - TREC Video Retrieval Evaluation,137,1352,TRECVID,TREC Video Retrieval Evaluation,NaN,NaN,NaN,TRECVID
14,ICADL - International Conference on Asian Digi...,604,1037,ICADL,International Conference on Asian Digital Libr...,NaN,NaN,NaN,ICADL


In [298]:
v[v['Shortname'] ==  'SIGIR']

,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,SIGIR - Research and Development in Informatio...,2717,72116,SIGIR,Research and Development in Information Retrieval,SIGIR,122,A1,SIGIR (A1)


In [299]:
v[v.Name.isnull()]

,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
9,Multimedia Information Retrieval,248,2847,MULTIMEDIA INFORMATION RETRIEVAL,None,NaN,NaN,NaN,MULTIMEDIA INFORMATION RETRIEVAL
11,Dublin Core Conference,65,1928,DUBLIN CORE CONFERENCE,None,NaN,NaN,NaN,DUBLIN CORE CONFERENCE
26,Information Retrieval,55,176,INFORMATION RETRIEVAL,None,NaN,NaN,NaN,INFORMATION RETRIEVAL
32,IR Workshop,1,98,IR WORKSHOP,None,NaN,NaN,NaN,IR WORKSHOP
34,Networked Information Retrieval,11,70,NETWORKED INFORMATION RETRIEVAL,None,NaN,NaN,NaN,NETWORKED INFORMATION RETRIEVAL
35,New Developments in Digital Libraries,34,68,NEW DEVELOPMENTS IN DIGITAL LIBRARIES,None,NaN,NaN,NaN,NEW DEVELOPMENTS IN DIGITAL LIBRARIES
37,ELSNET Summer School,8,15,ELSNET SUMMER SCHOOL,None,NaN,NaN,NaN,ELSNET SUMMER SCHOOL


In [336]:
# 14:35
v.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [337]:
v.iplot(kind='pie', labels='Estrato', values='Publications', pull=.1, hole=.0, textposition='inside')

# Generalization

In [317]:
qualis = pd.read_csv('data/qualis-2013-cs-all-venues.csv')
qualis.Sigla = qualis.Sigla.str.upper()

def read_subarea(filename):
    venues = pd.read_csv('data/' + filename, sep='\t')
    
    if 'Conferences' in venues: 
        column_name = 'Conferences'
    else:
        column_name = 'Journals'  # Are you sure? Qualis data here is only about Conferences
        
    venues[['Shortname', 'Name']] = venues[column_name].str.split(' - ', n=1, expand=True)
    venues.Shortname = venues.Shortname.str.upper()
    
    v = venues.merge(qualis, left_on='Shortname', right_on='Sigla', how='left')
    v = v.sort_values(by='Citation', ascending=False)
    v['ShortnameQualis'] = v.Shortname + ' (' + v.Estrato + ')'
    v.loc[v['ShortnameQualis'].isnull(), 'ShortnameQualis'] = v['Shortname']
    print('Found', len(v[v.Estrato.notnull()]), 'of', len(venues), 'venues')
    return v

In [318]:
filename = 'cs-networks-conf-10-years.tsv'
subarea_df = read_subarea(filename)
subarea_df.head()

Found 69 of 100 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,INFOCOM - IEEE INFOCOM,2562,23340,INFOCOM,IEEE INFOCOM,INFOCOM,204,A1,INFOCOM (A1)
1,ICC - IEEE International Conference on Communi...,3600,15312,ICC,IEEE International Conference on Communications,ICC,53,A2,ICC (A2)
2,SIGCOMM - ACM SIGCOMM Conference,367,9057,SIGCOMM,ACM SIGCOMM Conference,SIGCOMM,145,A1,SIGCOMM (A1)
3,MOBICOM - Mobile Computing and Networking,487,7881,MOBICOM,Mobile Computing and Networking,MOBICOM,112,A1,MOBICOM (A1)
4,PIMRC - IEEE International Symposium on Person...,3653,5976,PIMRC,"IEEE International Symposium on Personal, Indo...",PIMRC,60,A1,PIMRC (A1)


### All the files:

In [320]:
read_subarea('academic-multidisciplinary-journal-5-years.tsv').head()

Found 0 of 100 venues


,Journals,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,PNAS - Proceedings of The National Academy of ...,2796,1045,PNAS,Proceedings of The National Academy of Sciences,NaN,NaN,NaN,PNAS
1,Science,2323,812,SCIENCE,None,NaN,NaN,NaN,SCIENCE
2,PLOS One,17797,510,PLOS ONE,None,NaN,NaN,NaN,PLOS ONE
3,Nature,4687,274,NATURE,None,NaN,NaN,NaN,NATURE
4,NANO LETT - Nano Letters,682,68,NANO LETT,Nano Letters,NaN,NaN,NaN,NANO LETT


## Bioinformatics 

In [323]:
bio = read_subarea('cs-bioinformatics-conf-all-years.tsv')
bio.head()

Found 17 of 52 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,BioMED - Biomedical Engineering,2262,64723,BIOMED,Biomedical Engineering,NaN,NaN,NaN,BIOMED
1,ISMB - Intelligent Systems in Molecular Biology,795,27041,ISMB,Intelligent Systems in Molecular Biology,ISMB,91,A1,ISMB (A1)
2,Pacific Symposium on Biocomputing,791,15658,PACIFIC SYMPOSIUM ON BIOCOMPUTING,None,NaN,NaN,NaN,PACIFIC SYMPOSIUM ON BIOCOMPUTING
3,RECOMB - Research in Computational Molecular B...,883,13583,RECOMB,Research in Computational Molecular Biology,RECOMB,54,A2,RECOMB (A2)
4,CBMS - IEEE Symposium on Computer-Based Medica...,1845,4339,CBMS,IEEE Symposium on Computer-Based Medical Systems,CBMS,26,B1,CBMS (B1)


In [338]:
bio.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [324]:
bio10 = read_subarea('cs-bioinformatics-conf-10-years.tsv')
bio10.head()

Found 16 of 44 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,ISMB - Intelligent Systems in Molecular Biology,191,3002,ISMB,Intelligent Systems in Molecular Biology,ISMB,91,A1,ISMB (A1)
1,RECOMB - Research in Computational Molecular B...,360,1439,RECOMB,Research in Computational Molecular Biology,RECOMB,54,A2,RECOMB (A2)
2,Pacific Symposium on Biocomputing,300,1411,PACIFIC SYMPOSIUM ON BIOCOMPUTING,None,NaN,NaN,NaN,PACIFIC SYMPOSIUM ON BIOCOMPUTING
3,ICNSC - International Conference on Networking...,1192,1201,ICNSC,"International Conference on Networking, Sensin...",ICNSC,12,B3,ICNSC (B3)
4,CBMS - IEEE Symposium on Computer-Based Medica...,672,1025,CBMS,IEEE Symposium on Computer-Based Medical Systems,CBMS,26,B1,CBMS (B1)


In [339]:
bio10.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [ ]:
#v.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')
#v.iplot(kind='pie', labels='Estrato', values='Publications', pull=.1, hole=.0, textposition='inside')

In [351]:
bio10.iplot(kind='pie', labels='Estrato', values='Publications', pull=.1, hole=.0, textposition='inside')

In [327]:
data10 = read_subarea('cs-data-mining-conf-10-years.tsv')
data10.head()

Found 18 of 30 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,ICDE - International Conference on Data Engine...,1788,11903,ICDE,International Conference on Data Engineering,ICDE,113,A1,ICDE (A1)
1,KDD - Knowledge Discovery and Data Mining,817,9105,KDD,Knowledge Discovery and Data Mining,KDD,133,A1,KDD (A1)
2,CIKM - International Conference on Information...,1470,5355,CIKM,International Conference on Information and Kn...,CIKM,85,A1,CIKM (A1)
3,ICDM - IEEE International Conference on Data M...,1831,4645,ICDM,IEEE International Conference on Data Mining,ICDM,77,A1,ICDM (A1)
4,SDM - SIAM International Conference on Data Mi...,413,2329,SDM,SIAM International Conference on Data Mining,SDM,51,A2,SDM (A2)


In [348]:
data10.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [328]:
data = read_subarea('cs-data-mining-conf-all-years.tsv')
data.head()

Found 20 of 41 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,KDD - Knowledge Discovery and Data Mining,2063,69270,KDD,Knowledge Discovery and Data Mining,KDD,133,A1,KDD (A1)
1,ICDE - International Conference on Data Engine...,4012,67386,ICDE,International Conference on Data Engineering,ICDE,113,A1,ICDE (A1)
2,CIKM - International Conference on Information...,2636,28621,CIKM,International Conference on Information and Kn...,CIKM,85,A1,CIKM (A1)
3,ICDM - IEEE International Conference on Data M...,2506,18362,ICDM,IEEE International Conference on Data Mining,ICDM,77,A1,ICDM (A1)
4,SDM - SIAM International Conference on Data Mi...,708,9095,SDM,SIAM International Conference on Data Mining,SDM,51,A2,SDM (A2)


In [349]:
data.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [350]:
data10.iplot(kind='pie', labels='Estrato', values='Publications', pull=.1, hole=.0, textposition='inside')

In [329]:
ir10 = read_subarea('cs-ir-conf-10-years.tsv')
ir10.head()

Found 10 of 24 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,SIGIR - Research and Development in Informatio...,1014,9547,SIGIR,Research and Development in Information Retrieval,SIGIR,122,A1,SIGIR (A1)
1,CIKM - International Conference on Information...,1470,5355,CIKM,International Conference on Information and Kn...,CIKM,85,A1,CIKM (A1)
2,ECIR - European Colloquium on IR Research,490,1679,ECIR,European Colloquium on IR Research,ECIR,39,A2,ECIR (A2)
3,JCDL - ACM/IEEE Joint Conference on Digital Li...,519,1586,JCDL,ACM/IEEE Joint Conference on Digital Libraries,JCDL,53,A2,JCDL (A2)
4,Multimedia Information Retrieval,218,1501,MULTIMEDIA INFORMATION RETRIEVAL,None,NaN,NaN,NaN,MULTIMEDIA INFORMATION RETRIEVAL


In [340]:
ir10.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')
#v.iplot(kind='pie', labels='Estrato', values='Publications', pull=.1, hole=.0, textposition='inside')

In [330]:
ir = read_subarea('cs-ir-conf-all-years.tsv')
ir.head()

Found 10 of 41 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,SIGIR - Research and Development in Informatio...,2717,72116,SIGIR,Research and Development in Information Retrieval,SIGIR,122,A1,SIGIR (A1)
1,CIKM - International Conference on Information...,2636,28621,CIKM,International Conference on Information and Kn...,CIKM,85,A1,CIKM (A1)
2,TREC - Text REtrieval Conference,1433,23782,TREC,Text REtrieval Conference,TREC,51,A2,TREC (A2)
3,JCDL - ACM/IEEE Joint Conference on Digital Li...,1269,7203,JCDL,ACM/IEEE Joint Conference on Digital Libraries,JCDL,53,A2,JCDL (A2)
4,DL - Digital Libraries,351,6546,DL,Digital Libraries,DL,36,B1,DL (B1)


In [341]:
ir.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [332]:
net10 = read_subarea('cs-networks-conf-10-years.tsv')
net10.head()

Found 69 of 100 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,INFOCOM - IEEE INFOCOM,2562,23340,INFOCOM,IEEE INFOCOM,INFOCOM,204,A1,INFOCOM (A1)
1,ICC - IEEE International Conference on Communi...,3600,15312,ICC,IEEE International Conference on Communications,ICC,53,A2,ICC (A2)
2,SIGCOMM - ACM SIGCOMM Conference,367,9057,SIGCOMM,ACM SIGCOMM Conference,SIGCOMM,145,A1,SIGCOMM (A1)
3,MOBICOM - Mobile Computing and Networking,487,7881,MOBICOM,Mobile Computing and Networking,MOBICOM,112,A1,MOBICOM (A1)
4,PIMRC - IEEE International Symposium on Person...,3653,5976,PIMRC,"IEEE International Symposium on Personal, Indo...",PIMRC,60,A1,PIMRC (A1)


In [342]:
net10.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [333]:
net = read_subarea('cs-networks-conf-all-years.tsv')
net.head()

Found 55 of 100 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,INFOCOM - IEEE INFOCOM,6559,190527,INFOCOM,IEEE INFOCOM,INFOCOM,204,A1,INFOCOM (A1)
1,MOBICOM - Mobile Computing and Networking,940,83865,MOBICOM,Mobile Computing and Networking,MOBICOM,112,A1,MOBICOM (A1)
2,ICC - IEEE International Conference on Communi...,8283,78781,ICC,IEEE International Conference on Communications,ICC,53,A2,ICC (A2)
3,SIGCOMM - ACM SIGCOMM Conference,946,49960,SIGCOMM,ACM SIGCOMM Conference,SIGCOMM,145,A1,SIGCOMM (A1)
4,VTC - Vehicular Technology Conference,1084,49426,VTC,Vehicular Technology Conference,VTC,98,A1,VTC (A1)


In [343]:
net.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [344]:
net10.iplot(kind='pie', labels='Estrato', values='Publications', pull=.1, hole=.0, textposition='inside')

In [334]:
theory10 = read_subarea('cs-theory-conf-10-years.tsv')
theory10.head()

Found 49 of 101 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,SODA - ACM-SIAM Symposium on Discrete Algorithms,716,6718,SODA,ACM-SIAM Symposium on Discrete Algorithms,SODA,85,A1,SODA (A1)
1,STOC - ACM Symposium on Theory of Computing,347,5259,STOC,ACM Symposium on Theory of Computing,STOC,100,A1,STOC (A1)
2,"ICALP - International Colloquium on Automata, ...",497,2986,ICALP,"International Colloquium on Automata, Language...",ICALP,61,A1,ICALP (A1)
3,FOCS - IEEE Symposium on Foundations of Comput...,258,2878,FOCS,IEEE Symposium on Foundations of Computer Science,FOCS,94,A1,FOCS (A1)
4,ICCS - International Conference on Computation...,1795,2723,ICCS,International Conference on Computational Science,ICCS,37,A2,ICCS (A2)


In [345]:
theory10.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [335]:
theory = read_subarea('cs-theory-conf-all-years.tsv')
theory.head()

Found 45 of 100 venues


,Conferences,Publications,Citation,Shortname,Name,Sigla,Índice-­‐H,Estrato,ShortnameQualis
0,STOC - ACM Symposium on Theory of Computing,2884,121268,STOC,ACM Symposium on Theory of Computing,STOC,100,A1,STOC (A1)
1,FOCS - IEEE Symposium on Foundations of Comput...,3086,107338,FOCS,IEEE Symposium on Foundations of Computer Science,FOCS,94,A1,FOCS (A1)
2,SODA - ACM-SIAM Symposium on Discrete Algorithms,2333,50906,SODA,ACM-SIAM Symposium on Discrete Algorithms,SODA,85,A1,SODA (A1)
3,"ICALP - International Colloquium on Automata, ...",2510,37224,ICALP,"International Colloquium on Automata, Language...",ICALP,61,A1,ICALP (A1)
4,LICS - Logic in Computer Science,1345,36850,LICS,Logic in Computer Science,LICS,52,A2,LICS (A2)


In [346]:
theory.iplot(kind='pie', labels='ShortnameQualis', values='Publications', pull=.2, hole=.1, textposition='inside')

In [352]:
theory10.iplot(kind='pie', labels='Estrato', values='Publications', pull=.1, hole=.0, textposition='inside')